## Application Package reproducibility

### Personas

* **Alice** developed a Water Body detection Earth Observation application and package it as an EO Application Package
* **Bob** scripts the execution of application

### Scenario

Alice included in the water bodies detection Application Package software repository a Continuous Integration configuration relying on Github Actions to:

* build the containers
* push the built containers to Github container registry
* update the Application Package with these new container references
* push the updated Application Package to Github's artifact registry


Alice sent an email to Bob:

<hr>
from: alice@acme.io

to: bob@acme.io

subject: Detecting water bodies with NDWI and the Otsu threshold


Hi Bob!

checkout my new application package for detecting water bodies using NDWI and the Ostu threshold.

I've ran it over our test site bounding box and prelimanry result look promising.

The github repo is https://github.com/Terradue/app-package-training-bids23 and I've just released version 1.0.0.

Let me know!

Cheers

Alice
<hr>

With this information, Bob scripts the Application Execution in a Jupyter Notebook.

His environment has a container engine (e.g. podman or docker) and the cwltool CWL runner.

## Running the Scenario

In [ ]:
import argparse
import asyncio
import json
import os
from datetime import datetime
from io import StringIO

import nest_asyncio
import pystac
import rasterio
from cwltool.main import main
from ipyleaflet import GeoJSON, Map
from pydantic_yaml import to_yaml_str
from pystac_client import Client
from rasterio.features import dataset_features, sieve

from helpers import Params, get_param_model_fields, get_release_assets, stage_in

from shutil import which

nest_asyncio.apply()

## Check the container engine

In [ ]:
if which("podman"):
    podman = True
elif which("docker"):
    podman = False
else:
    raise ValueError("No container engine")

## Application Package releases

Bob uses Github API to list the artifacts published by Alice in the release

In [ ]:
assets = get_release_assets(
    user="Terradue",
    repo="app-package-training-bids23",
    token=os.environ["GH_PAT"],
)

assets

## Running the Application Package to detect water bodies on Sentinel-2 data

Alice published three Application Packages.

 Bob selects the one processing several Sentinel-2 acquisitions provided as STAC Items


In [ ]:
app_package = assets["1.0.0"][0]

print(app_package["doc"])

print(app_package["url"])

The Application Package parameters are discovered and a pydantic model is created

In [ ]:
Params.set_fields(**get_param_model_fields(cwl_obj=app_package["cwl"]))

Params.get_fields()

The Application Package takes as inputs:
- one or more STAC Items
- a list of the bands for the normalized difference
- an area of interest
- the EPSG code used for the area of interest coordinates

 Bob uses a STAC API endpoint to discover Sentinel-2 acquisitions over an area of interest and time of interest 

In [ ]:
URL = "https://earth-search.aws.element84.com/v1/"

headers = []

cat = Client.open(URL, headers=headers)
cat

Bod defines the search parameter and get the results:

In [ ]:
# Collection
collections = ["sentinel-2-l2a"]

# Start and end dates
start_date = datetime.fromisoformat("2021-07-08T00:00:00")
stop_date = datetime.fromisoformat("2021-07-08T23:59:59")

bbox = [-121.399, 39.834, -120.74, 40.472]

# Other metadata
cloud_cover = 5

# Query by AOI, start and end date and other params
query = cat.search(
    collections=collections,
    datetime=(start_date, stop_date),
    bbox=bbox,
    query={"eo:cloud_cover": {"lt": cloud_cover}},
)

Bob plots the Sentinel-2 discovered STAC Items footprint:

In [ ]:
center = ((bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2)

m = Map(center=center, zoom=8)

for item in list(query.item_collection()):
    geo_json = GeoJSON(
        name=item.id,
        data=item.geometry,
        style={
            "opacity": 1,
            "dashArray": "9",
            "fillOpacity": 0.1,
            "weight": 1,
            "color": "blue",
        },
        hover_style={"color": "white", "dashArray": "0", "fillOpacity": 0.5},
    )
    m.add_layer(geo_json)

m

Bob lists the STAC Items self link, these are the URLs to the Sentinel-2 STAC Items to process:

In [ ]:
[item.get_self_href() for item in list(query.item_collection())]

And creates the parameters for running the Application Package (the epsg and bands input parameters have default values)

In [ ]:
params = Params(
    aoi=",".join([str(elem) for elem in bbox]),
    stac_items=[item.self_href for item in query.item_collection()],
    epsg="EPSG:4326",
    bands=["green", "nir"],
)

params.dict()

Bob writes a YAML file with the parameters and their values:

In [ ]:
with open("params-s2.yaml", "w") as file:
    print(to_yaml_str(params), file=file)

The file `params.yaml` contains:

```yaml
aoi: -121.399,39.834,-120.74,40.472
bands:
- green
- nir
epsg: EPSG:4326
stac_items:
- https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20210708_0_L2A
- https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20210708_1_L2A
```

Bob uses the CWL runner `cwltool` Python API to script the Application Package execution 

In [ ]:
parsed_args = argparse.Namespace(
    podman=podman,
    parallel=True,
    debug=False,
    outdir="./runs",
    workflow=app_package["url"],
    job_order=["params-s2.yaml"],
)

stream_out = StringIO()
stream_err = StringIO()

res = main(
    args=parsed_args,
    stdout=stream_out,
)

assert res == 0

This execution generates as output a JSON file listing all files produced.

The JSON contains the output defined in the CWL workflow that can be accessed with: 

```python
os.path.basename(app_package["cwl"].outputs[0].id)
```

In [ ]:
results = json.loads(stream_out.getvalue())

results[os.path.basename(app_package["cwl"].outputs[0].id)]

Bob writes a simple code to find the STAC Catalog path and then list the contents of that STAC Catalog:

In [ ]:
cat = pystac.read_file(
    [
        listing["path"]
        for listing in results[os.path.basename(app_package["cwl"].outputs[0].id)][
            "listing"
        ]
        if "catalog.json" in listing["path"]
    ][0]
)

cat.describe()

Bob uses the STAC Python library to open the first STAC Item produced:

In [ ]:
item = next(cat.get_items())
item

Bob gets the path of the ostu step asset:

In [ ]:
asset_href = item.get_assets()["data"].get_absolute_href()

asset_href

Bob applies the sieve algorithm and then vectorizes the water bodies.

Finally the water bodies are added to a map

In [ ]:
# Define the threshold size to remove small features (in pixels)
threshold = 100  # Adjust this threshold as needed
connectivity = 4  # Use 4-connected pixels for the sieve operation

center = ((bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2)

m = Map(center=center, zoom=8)

with rasterio.open(asset_href) as src:
    result = sieve(src, threshold, connectivity=8)
    for geom in dataset_features(src, band=True, as_mask=True):
        geo_json = GeoJSON(
            name="",
            data=geom,
            style={
                "opacity": 1,
                "fillOpacity": 0.1,
                "weight": 1,
                "color": "red",
            },
            hover_style={"color": "red", "dashArray": "0", "fillOpacity": 0.5},
        )
        m.add_layer(geo_json)
m

## Landsat-8

Bob now applies the Water Bodies application package to a Landsat-9 acquisition.

Bob uses a STAC Catalog with the `landsat-c2-l2` collection

In [ ]:
URL = "https://planetarycomputer.microsoft.com/api/stac/v1"

headers = []

cat = Client.open(URL, headers=headers, ignore_conformance=True)
cat

In [ ]:
# Collection
collections = ["landsat-c2-l2"]

# Start and end dates
start_date = datetime.fromisoformat("2023-07-01T00:00:00")
stop_date = datetime.fromisoformat("2023-07-01T23:59:59")

# bbox = [-121.399, 39.834, -120.74, 40.472]
bbox = [-121.503, 40.049, -120.509, 40.835]

# Other metadata
cloud_cover = 5

# Define EPSG code
epsg = "EPSG:4326"

# Query by AOI, start and end date and other params
query = cat.search(
    collections=collections,
    datetime=(start_date, stop_date),
    bbox=bbox,
    query={"eo:cloud_cover": {"lt": cloud_cover}},
)

In [ ]:
[item.get_self_href() for item in list(query.item_collection())]

Bob uses the stac-asset python library to stage the Landsat acquisition:

In [ ]:
item = [item for item in list(query.item_collection())][0]

print(item)

staged_catalog = asyncio.run(stage_in(item, target_dir="data"))

The staged Landsat product is a STAC Item in a STAC Catalog:

In [ ]:
staged_catalog.describe()

In [ ]:
staged_item = next(staged_catalog.get_items())

staged_item

In [ ]:
staged_item.get_assets()["green"]

In [ ]:
staged_item.get_assets()["nir08"]

In [ ]:
app_package = assets["1.0.0"][2]

print(app_package["doc"])

print(app_package["url"])

In [ ]:
Params.set_fields(**get_param_model_fields(cwl_obj=app_package["cwl"]))

Params.get_fields()

In [ ]:
params = Params(
    aoi=",".join([str(elem) for elem in bbox]),
    item=os.path.dirname(staged_catalog.self_href),
    bands=["green", "nir08"],
)

In [ ]:
params

In [ ]:
with open("params-ls9.yaml", "w") as file:
    print(to_yaml_str(params), file=file)

In [ ]:
parsed_args = argparse.Namespace(
    podman=False,
    parallel=True,
    outdir="./runs",
    workflow=app_package["url"],
    job_order=["params-ls9.yaml"],
)

stream_out = StringIO()
stream_err = StringIO()

res = main(
    args=parsed_args,
    stdout=stream_out,
)

assert res == 0

In [ ]:
results = json.loads(stream_out.getvalue())

cat = pystac.read_file(
    [
        listing["path"]
        for listing in results[os.path.basename(app_package["cwl"].outputs[0].id)][
            "listing"
        ]
        if "catalog.json" in listing["path"]
    ][0]
)

cat.describe()

In [ ]:
item = next(cat.get_items())
asset = item.get_assets()["data"].get_absolute_href()

asset

In [ ]:
center = ((bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2)

m = Map(center=center, zoom=8)

# Define the threshold size to remove small features (in pixels)
threshold = 100  # Adjust this threshold as needed
connectivity = 8  # Use 4-connected pixels for the sieve operation

with rasterio.open(asset) as src:
    result = sieve(src, threshold, connectivity=connectivity)
    for index, geom in enumerate(dataset_features(src, band=True, as_mask=True)):
        geo_json = GeoJSON(
            name="",
            data=geom,
            style={
                "opacity": 1,
                "fillOpacity": 0.1,
                "weight": 1,
                "color": "red",
            },
            hover_style={"color": "red", "dashArray": "0", "fillOpacity": 0.5},
        )
        m.add_layer(geo_json)
        if index == 1000:
            break
m

Bob sends an email to Alice

<hr>
from: bob@acme.io

to: alice@acme.io

subject: RE:Detecting water bodies with NDWI and the Otsu threshold


Hi Alice!

The results look promising with Sentinel-2 data. 
I've ran the Application Package against Landsat-9 data and we should discuss why there so many false positives.

Cheers,

Bob
<hr>